<a href="https://colab.research.google.com/github/sungjin417/sesac-phython/blob/master/%EA%B5%90)RAG%EA%B8%B0%EB%B2%95%EC%9D%98_%EC%9D%B4%ED%95%B4%EC%99%80_%EC%A0%81%EC%9A%A9_1%EC%9D%BC%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 🌼 RAG기법의 이해와 적용 - 1차시(24.11.28)

---

In [1]:
# pip install langchain_community

In [2]:
from langchain_community.document_loaders import RecursiveUrlLoader # 자동으로 크롤링
DOCS_PAGE='https://blog.vllm.ai/2023/06/20/vllm.html'

In [3]:
loader = RecursiveUrlLoader(DOCS_PAGE)

In [4]:
docs = loader.load()

In [5]:
docs

[Document(metadata={'source': 'https://blog.vllm.ai/2023/06/20/vllm.html', 'content_type': 'text/html; charset=utf-8', 'title': 'vLLM: Easy, Fast, and Cheap LLM Serving with PagedAttention | vLLM Blog', 'description': 'GitHub | Documentation | Paper', 'language': 'en'}, page_content='<!DOCTYPE html>\n<html lang="en"><head>\n  <meta charset="utf-8">\n  <meta http-equiv="X-UA-Compatible" content="IE=edge">\n  <meta name="viewport" content="width=device-width, initial-scale=1"><!-- Begin Jekyll SEO tag v2.8.0 -->\n<title>vLLM: Easy, Fast, and Cheap LLM Serving with PagedAttention | vLLM Blog</title>\n<meta name="generator" content="Jekyll v4.3.3" />\n<meta property="og:title" content="vLLM: Easy, Fast, and Cheap LLM Serving with PagedAttention" />\n<meta name="author" content="Woosuk Kwon*, Zhuohan Li*, Siyuan Zhuang, Ying Sheng, Lianmin Zheng, Cody Yu, Joey Gonzalez, Hao Zhang, and Ion Stoica (* Equal Contribution)" />\n<meta property="og:locale" content="en_US" />\n<meta name="descripti

In [6]:
num_documents = len(docs)
print(f'document 개수 : {num_documents}')

document 개수 : 1


In [7]:
import torch
from sentence_transformers import SentenceTransformer
N_GPU = torch.cuda.device_count()

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [9]:
model_name = "BAAI/bge-large-en-v1.5"

In [10]:
encoder = SentenceTransformer(model_name, device = device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [11]:
embedding_dim = encoder.get_sentence_embedding_dimension()
max_seq_length = encoder.get_max_seq_length()

In [12]:
print(f'문장 임베딩 시 생성되는 벡터의 차원 크기 : {embedding_dim}')
print(f'모델이 처리할 수 있는 입력 문장의 최대 토큰 길이 : {max_seq_length}')

문장 임베딩 시 생성되는 벡터의 차원 크기 : 1024
모델이 처리할 수 있는 입력 문장의 최대 토큰 길이 : 512


In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import numpy as np

chunk_size = 512
chunk_overlap = np.round(chunk_size * 0.1, 0) # 오차범위를 두어 겹치게 하는 거(문맥이 중간에 짤리면 내용이 이어지지 않기 때문에)
# 10 ~ 20 % 가 적당 너무 짧으면 여전이 문맥이 끊기고, 너무 길어도 문제가 생기기 때문에
print(f'chunk_size : {chunk_size}, chunk_overlap : {chunk_overlap}')

chunk_size : 512, chunk_overlap : 51.0


In [14]:
# chunk_overlap을 이해하기 위한 예시
long_text = '안녕하세요 아주 긴 텍스트를 나눠보겠습니다. 문장 단위로 어떻게 나누어야 할까요?'
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 20,
    chunk_overlap = 5

)
text_chunks = text_splitter.split_text(long_text)
for chunk in text_chunks:
  print(chunk)

안녕하세요 아주 긴 텍스트를
텍스트를 나눠보겠습니다. 문장
문장 단위로 어떻게 나누어야
나누어야 할까요?


In [15]:
child_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_size,
    chunk_overlap = chunk_overlap
)

In [16]:
chunks = child_splitter.split_documents(docs)
print(f'{len(docs)}개의 docs가 {len(chunks)}개의 문자로 나뉘었다') # docs는 보통 1개가 아니라 여러개이다 참고!!!!!

1개의 docs가 52개의 문자로 나뉘었다


In [17]:
string_list = []
for doc in chunks:
  if hasattr(doc, 'page_content'): # page_content를 확인하는 코드
  # hasattr(객체, 속성명) -> 객체에 속성명이 있는지
    string_list.append(doc.page_content)

In [18]:
string_list # page_content의 값만 담김!!!!!

['<!DOCTYPE html>\n<html lang="en"><head>\n  <meta charset="utf-8">\n  <meta http-equiv="X-UA-Compatible" content="IE=edge">\n  <meta name="viewport" content="width=device-width, initial-scale=1"><!-- Begin Jekyll SEO tag v2.8.0 -->\n<title>vLLM: Easy, Fast, and Cheap LLM Serving with PagedAttention | vLLM Blog</title>\n<meta name="generator" content="Jekyll v4.3.3" />\n<meta property="og:title" content="vLLM: Easy, Fast, and Cheap LLM Serving with PagedAttention" />',
 '<meta name="author" content="Woosuk Kwon*, Zhuohan Li*, Siyuan Zhuang, Ying Sheng, Lianmin Zheng, Cody Yu, Joey Gonzalez, Hao Zhang, and Ion Stoica (* Equal Contribution)" />\n<meta property="og:locale" content="en_US" />\n<meta name="description" content="GitHub | Documentation | Paper" />\n<meta property="og:description" content="GitHub | Documentation | Paper" />\n<meta property="og:site_name" content="vLLM Blog" />\n<meta property="og:type" content="article" />',
 '<meta property="og:type" content="article" />\n<me

In [19]:
embeddings = torch.tensor(encoder.encode(string_list))
embeddings

tensor([[-0.0234,  0.0221, -0.0177,  ..., -0.0006, -0.0043,  0.0399],
        [ 0.0171,  0.0109, -0.0180,  ..., -0.0039, -0.0265,  0.0146],
        [ 0.0079,  0.0002,  0.0203,  ..., -0.0080, -0.0263,  0.0426],
        ...,
        [ 0.0074,  0.0103,  0.0246,  ...,  0.0150, -0.0150,  0.0111],
        [-0.0002,  0.0002,  0.0041,  ...,  0.0242, -0.0120, -0.0135],
        [-0.0334, -0.0142,  0.0187,  ..., -0.0048, -0.0167,  0.0039]])

In [20]:
# 얼마나 가까운지, 얼마나 유사한지 판단하기 위해 정규화를 진행 해줘야 함
embeddings = np.array(embeddings / np.linalg.norm(embeddings))
# vector 정규화 : 상대적인 값을 유지하며 크기를 조정하여 유사도, 거리 등을 비교하기 위해
embeddings

array([[-3.2433148e-03,  3.0656536e-03, -2.4589787e-03, ...,
        -7.9131096e-05, -5.9362175e-04,  5.5376114e-03],
       [ 2.3663254e-03,  1.5079604e-03, -2.4914653e-03, ...,
        -5.3539872e-04, -3.6686282e-03,  2.0307114e-03],
       [ 1.0944124e-03,  3.0741558e-05,  2.8103061e-03, ...,
        -1.1119934e-03, -3.6409216e-03,  5.9099812e-03],
       ...,
       [ 1.0273707e-03,  1.4269428e-03,  3.4093512e-03, ...,
         2.0777669e-03, -2.0858389e-03,  1.5358157e-03],
       [-3.2893062e-05,  2.6786507e-05,  5.6232640e-04, ...,
         3.3581476e-03, -1.6645045e-03, -1.8742131e-03],
       [-4.6384879e-03, -1.9680921e-03,  2.5938142e-03, ...,
        -6.6451193e-04, -2.3227520e-03,  5.3908327e-04]], dtype=float32)

In [21]:
converted_values = list(map(np.float32, embeddings)) # np.float32 형태로 변환 해줘
converted_values

[array([-3.2433148e-03,  3.0656536e-03, -2.4589787e-03, ...,
        -7.9131096e-05, -5.9362175e-04,  5.5376114e-03], dtype=float32),
 array([ 0.00236633,  0.00150796, -0.00249147, ..., -0.0005354 ,
        -0.00366863,  0.00203071], dtype=float32),
 array([ 1.0944124e-03,  3.0741558e-05,  2.8103061e-03, ...,
        -1.1119934e-03, -3.6409216e-03,  5.9099812e-03], dtype=float32),
 array([ 1.4564579e-03,  4.7587943e-03, -1.1864128e-03, ...,
         4.6959500e-05, -3.8631826e-03,  4.2156610e-03], dtype=float32),
 array([-0.00234334,  0.00301518, -0.00062454, ..., -0.00045515,
        -0.00390585,  0.00814049], dtype=float32),
 array([ 0.00089627,  0.00172723, -0.00284757, ..., -0.00452556,
        -0.00457108,  0.00146858], dtype=float32),
 array([ 0.00058679,  0.00072843,  0.00045178, ..., -0.00290517,
        -0.00711539,  0.00377408], dtype=float32),
 array([ 0.00090235, -0.00144551,  0.003178  , ..., -0.00347104,
        -0.00448765, -0.00245557], dtype=float32),
 array([-0.0028666

In [22]:
dict_list = []
for chunk, vector in zip(chunks,converted_values):
  # chunk : 현재 테스트 조각
  # vector : 해당 테스트 조각의 임베딩 벡터
  chunk_dict = {
      'chunk' : chunk.page_content,
      'source' : chunk.metadata.get('source',''), # 없으면 빈 배열 값 저장
      'vector' : vector
  }
  dict_list.append(chunk_dict)

In [23]:
dict_list

[{'chunk': '<!DOCTYPE html>\n<html lang="en"><head>\n  <meta charset="utf-8">\n  <meta http-equiv="X-UA-Compatible" content="IE=edge">\n  <meta name="viewport" content="width=device-width, initial-scale=1"><!-- Begin Jekyll SEO tag v2.8.0 -->\n<title>vLLM: Easy, Fast, and Cheap LLM Serving with PagedAttention | vLLM Blog</title>\n<meta name="generator" content="Jekyll v4.3.3" />\n<meta property="og:title" content="vLLM: Easy, Fast, and Cheap LLM Serving with PagedAttention" />',
  'source': 'https://blog.vllm.ai/2023/06/20/vllm.html',
  'vector': array([-3.2433148e-03,  3.0656536e-03, -2.4589787e-03, ...,
         -7.9131096e-05, -5.9362175e-04,  5.5376114e-03], dtype=float32)},
 {'chunk': '<meta name="author" content="Woosuk Kwon*, Zhuohan Li*, Siyuan Zhuang, Ying Sheng, Lianmin Zheng, Cody Yu, Joey Gonzalez, Hao Zhang, and Ion Stoica (* Equal Contribution)" />\n<meta property="og:locale" content="en_US" />\n<meta name="description" content="GitHub | Documentation | Paper" />\n<meta p



```
chunks = [
  Document(page_content = '첫 번째 본문', metadata = {'source' : 'doc1.com'}),
  Document(page_content = '두 번째 본문', metadata = {'source' : 'doc2.com'})
]

converted_values = [
  [0.1, 0.2, 0.3],
  [0.4, 0.5, 0.6]
]
```





```
{
  'chunk' : '첫 번째 본문',
  'source' : 'doc1.com',
  'vector' : [0.1, 0.2, 0.3]
}


```



In [24]:
# pip install pymilvus # 벡터 디비를 저장하는 디비이름

In [25]:
from pymilvus import MilvusClient # 리누스환경에서 사용 가능!!
import time

In [26]:
mc = MilvusClient('milvus_test.db') # 형식 milvus(일반적으로 붙임)_이름.db

DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: 3c0a6edc927145fa92cd4d6b04c3635c


In [27]:
collection_name = 'milvus_docs'
# CRUD : Create, Read, Update, Delete

In [28]:
mc.create_collection(collection_name, # collection_name = 'milvus_docs'
                     embedding_dim, # 벡터의 차원값을 반환 받은 값
                     consustency_level = 'Eventually', # 데이터의 일관성 수준(실시간의 최신 데이터를 처리해야 하는 지의 문제)
                     # 일반적(검색 등) : 'Eventually'
                     # 강하게(주식등) : Strong
                     auto_id = True, # index 자동 생성
                     overwrite = True) # 덮어쓰기(데이터 최신화)
                     # overwirte : 이미 존재하는 컬렉션이 있으면 덮어쓰기

DEBUG:pymilvus.milvus_client.milvus_client:Successfully created collection: milvus_docs
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created an index on collection: milvus_docs


In [29]:
print('데이터 삽입 시작!!!')
start_time = time.time()

mc.insert(
    collection_name,
    data = dict_list,
    progress_bar = True
)

end_time = time.time()
print(f'삽입된 벡터(데이터)의 개수 : {len(dict_list)}', end='')
print(f'삽입에 걸린 시간 : {round(end_time - start_time, 2)} 초')

데이터 삽입 시작!!!
삽입된 벡터(데이터)의 개수 : 52삽입에 걸린 시간 : 0.03 초


In [30]:
import torch.nn.functional as F

In [31]:
sample_question = 'What is PagedAttention?'

In [32]:
query_embeddings = torch.tensor(encoder.encode([sample_question]))

In [33]:
query_embeddings

tensor([[-0.0537, -0.0164,  0.0005,  ..., -0.0165, -0.0059,  0.0475]])

In [34]:
query_embddings = F.normalize(query_embeddings, p = 2, dim = 1)  # 하나의 행을 정규화 할때 사용가능
# p : 정규화 할때 사용할 벡터의 길이
# dim : 차원(축)을 기준으로 할지 결정
query_embeddings

tensor([[-0.0537, -0.0164,  0.0005,  ..., -0.0165, -0.0059,  0.0475]])

In [35]:
query_embeddings = list(map(np.float32, query_embeddings)) # array 형태로 저장
query_embeddings

[array([-0.0537494 , -0.01643682,  0.00046268, ..., -0.01646634,
        -0.00591078,  0.04753314], dtype=float32)]

In [36]:
output_fields = list(dict_list[0].keys())
output_fields

['chunk', 'source', 'vector']

In [37]:
# vector 값은 필요 없기 때문에 제거
output_fields.remove('vector')

In [38]:
top_k = 4

In [39]:
result = mc.search(
    collection_name,
    data = query_embeddings,
    output_fields = output_fields,
    limit = top_k,
    consistency_level = 'Eventually'
)

In [40]:
result

data: ['[{\'id\': 454237455411314716, \'distance\': 0.780035138130188, \'entity\': {\'chunk\': \'<p>PagedAttention naturally enables memory sharing through its block table. Similar to how processes share physical pages, different sequences in PagedAttention can share the blocks by mapping their logical blocks to the same physical block. To ensure safe sharing, PagedAttention keeps track of the reference counts of the physical blocks and implements the <em>Copy-on-Write</em> mechanism.</p>\', \'source\': \'https://blog.vllm.ai/2023/06/20/vllm.html\'}}, {\'id\': 454236856952553604, \'distance\': 0.780035138130188, \'entity\': {\'chunk\': \'<p>PagedAttention naturally enables memory sharing through its block table. Similar to how processes share physical pages, different sequences in PagedAttention can share the blocks by mapping their logical blocks to the same physical block. To ensure safe sharing, PagedAttention keeps track of the reference counts of the physical blocks and implements

In [41]:
result[0][0]['entity']['chunk']

'<p>PagedAttention naturally enables memory sharing through its block table. Similar to how processes share physical pages, different sequences in PagedAttention can share the blocks by mapping their logical blocks to the same physical block. To ensure safe sharing, PagedAttention keeps track of the reference counts of the physical blocks and implements the <em>Copy-on-Write</em> mechanism.</p>'

In [42]:
result[0][1]['entity']['chunk']

'<p>PagedAttention naturally enables memory sharing through its block table. Similar to how processes share physical pages, different sequences in PagedAttention can share the blocks by mapping their logical blocks to the same physical block. To ensure safe sharing, PagedAttention keeps track of the reference counts of the physical blocks and implements the <em>Copy-on-Write</em> mechanism.</p>'

In [43]:
result[0][2]['entity']['chunk']

'<p>PagedAttention naturally enables memory sharing through its block table. Similar to how processes share physical pages, different sequences in PagedAttention can share the blocks by mapping their logical blocks to the same physical block. To ensure safe sharing, PagedAttention keeps track of the reference counts of the physical blocks and implements the <em>Copy-on-Write</em> mechanism.</p>'

In [44]:
result[0][3]['entity']['chunk']

'<p>PagedAttention is the core technology behind vLLM, our LLM inference and serving engine that supports a variety of models with high performance and an easy-to-use interface. For more technical details about vLLM and PagedAttention, check out our <a href="https://github.com/vllm-project/vllm">GitHub repo</a> and stay tuned for our paper.</p>\n\n<h3 id="the-silent-hero-behind-lmsys-vicuna-and-chatbot-arena">The Silent Hero Behind LMSYS Vicuna and Chatbot Arena</h3>'

In [45]:
# # 가장 유사도 높은 하나의 단일 chunk만 추가하는 경우
# contexts = []
# # 검색을 해서 찾아온거지 모델이 대답한게 아니기 때문에..
# contexts.append(result[0][0]['entity']['chunk'])

In [46]:
# contexts

['<p>PagedAttention naturally enables memory sharing through its block table. Similar to how processes share physical pages, different sequences in PagedAttention can share the blocks by mapping their logical blocks to the same physical block. To ensure safe sharing, PagedAttention keeps track of the reference counts of the physical blocks and implements the <em>Copy-on-Write</em> mechanism.</p>']

In [47]:
# top_k 의 chunk를 모두 추가하는 경우
contexts = []
for i in range(top_k):
  contexts.append(result[0][i]['entity']['chunk'])
contexts

['<p>PagedAttention naturally enables memory sharing through its block table. Similar to how processes share physical pages, different sequences in PagedAttention can share the blocks by mapping their logical blocks to the same physical block. To ensure safe sharing, PagedAttention keeps track of the reference counts of the physical blocks and implements the <em>Copy-on-Write</em> mechanism.</p>',
 '<p>PagedAttention naturally enables memory sharing through its block table. Similar to how processes share physical pages, different sequences in PagedAttention can share the blocks by mapping their logical blocks to the same physical block. To ensure safe sharing, PagedAttention keeps track of the reference counts of the physical blocks and implements the <em>Copy-on-Write</em> mechanism.</p>',
 '<p>PagedAttention naturally enables memory sharing through its block table. Similar to how processes share physical pages, different sequences in PagedAttention can share the blocks by mapping the

In [48]:
contexts_combined = ' '.join(contexts)
contexts_combined

'<p>PagedAttention naturally enables memory sharing through its block table. Similar to how processes share physical pages, different sequences in PagedAttention can share the blocks by mapping their logical blocks to the same physical block. To ensure safe sharing, PagedAttention keeps track of the reference counts of the physical blocks and implements the <em>Copy-on-Write</em> mechanism.</p> <p>PagedAttention naturally enables memory sharing through its block table. Similar to how processes share physical pages, different sequences in PagedAttention can share the blocks by mapping their logical blocks to the same physical block. To ensure safe sharing, PagedAttention keeps track of the reference counts of the physical blocks and implements the <em>Copy-on-Write</em> mechanism.</p> <p>PagedAttention naturally enables memory sharing through its block table. Similar to how processes share physical pages, different sequences in PagedAttention can share the blocks by mapping their logica

In [49]:
system_prompt = f"""유사도가 높게 나온 Context : {contexts_combined}"""
prompts = [system_prompt]
prompts

['유사도가 높게 나온 Context : <p>PagedAttention naturally enables memory sharing through its block table. Similar to how processes share physical pages, different sequences in PagedAttention can share the blocks by mapping their logical blocks to the same physical block. To ensure safe sharing, PagedAttention keeps track of the reference counts of the physical blocks and implements the <em>Copy-on-Write</em> mechanism.</p> <p>PagedAttention naturally enables memory sharing through its block table. Similar to how processes share physical pages, different sequences in PagedAttention can share the blocks by mapping their logical blocks to the same physical block. To ensure safe sharing, PagedAttention keeps track of the reference counts of the physical blocks and implements the <em>Copy-on-Write</em> mechanism.</p> <p>PagedAttention naturally enables memory sharing through its block table. Similar to how processes share physical pages, different sequences in PagedAttention can share the blocks b

In [50]:
from huggingface_hub import login
from google.colab import userdata
hf_token = input('HF')
login(token = hf_token)

HFhf_CZGDjjpoyCySVNIEpvkqOSnHnqwlbsWjPT


In [51]:
# pip install vllm

In [54]:
# pip install triton

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 4.9 MB/s eta 0:00:00


In [55]:
import vllm, torch
from vllm import LLM, SamplingParams

In [56]:
import torch
model_torun = "meta-llama/Llama-3.2-1B-Instruct"

In [57]:
# GPU 메모리 캐시 지우기
torch.cuda.empty_cache()

In [60]:
llm = LLM(model = model_torun,
          enforce_eager = True,
          dtype = torch.float16,
          gpu_memory_utilization = 0.9,
          max_model_len = 1024)

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

WARNING 11-28 07:34:21 config.py:1865] Casting torch.bfloat16 to torch.float16.
INFO 11-28 07:34:31 config.py:350] This model supports multiple tasks: {'embedding', 'generate'}. Defaulting to 'generate'.
WARNING 11-28 07:34:31 config.py:503] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 11-28 07:34:31 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='meta-llama/Llama-3.2-1B-Instruct', speculative_config=None, tokenizer='meta-llama/Llama-3.2-1B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=1024, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=c

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

...

INFO 11-28 07:34:34 selector.py:261] Cannot use FlashAttention-2 backend for Volta and Turing GPUs.
INFO 11-28 07:34:34 selector.py:144] Using XFormers backend.
INFO 11-28 07:34:35 model_runner.py:1072] Starting to load model meta-llama/Llama-3.2-1B-Instruct...
INFO 11-28 07:34:35 weight_utils.py:243] Using model weights format ['*.safetensors']


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

INFO 11-28 07:35:36 weight_utils.py:288] No model.safetensors.index.json found in remote.


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 11-28 07:35:41 model_runner.py:1077] Loading model weights took 2.3185 GB
INFO 11-28 07:35:41 worker.py:232] Memory profiling results: total_gpu_memory=14.75GiB initial_memory_usage=3.74GiB peak_torch_memory=4.75GiB memory_usage_post_profile=3.76GiB non_torch_memory=0.18GiB kv_cache_size=8.34GiB gpu_memory_utilization=0.90
INFO 11-28 07:35:41 gpu_executor.py:113] # GPU blocks: 17090, # CPU blocks: 8192
INFO 11-28 07:35:41 gpu_executor.py:117] Maximum concurrency for 1024 tokens per request: 267.03x


In [62]:
from vllm import SamplingParams
sampling_params = SamplingParams(temperature = 0.2, top_p = 0.95)

In [65]:
outputs = llm.generate(prompts, sampling_params)

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.62it/s, est. speed input: 969.37 toks/s, output: 42.37 toks/s]


In [67]:
for output in outputs:
  prompt = output.prompt
  generated_text = output.outputs[0].text
  print(f'질문 : {sample_question}')
  print(f'생성된 모델의 응답 : {generated_text}')

질문 : What is PagedAttention?
생성된 모델의 응답 :  <p>Our LLM inference and serving engine, vLLM, is
